# patent가 인용한 patent 찾기

### 추출할 element
patent_id: 첫 doc-number  
pat_cited: citation / patcit / document-id / country='US' & len(doc-number) == 7 => **country != 'US'에서 탈출** (다음 doc으로 ㄱㄱ)   
examiner: citation / category

### some issues on the elements
**patent_id**
- 맨 앞이 D로 시작하는 애들 때문에 D없는 애들 앞에 0이 붙음   
- but, 우리가 관심있는 특허는 D로 시작하는 애들 없음  
- **=> int 만들어서 0 떼고 value Error 뜨면 바로 다음 doc으로 넘겨버리자**
  
**examiner**  
- dummy로 변경

In [1]:
#-*- coding: utf-8 -*-
import os
import re
import csv
import sys
import xml.etree.ElementTree as et
import datetime as dt

os.chdir('E:/apps')

In [2]:
def extract_xml_strings(filename):
    """
    Given a string [filename], opens the file and returns a generator
    that yields tuples. A tuple is of format (year, xmldoc string). A tuple
    is returned for every valid XML doc in [filename]
    """
    # search for terminating XML tag
    endtag_regex = re.compile('^<!DOCTYPE (.*) SYSTEM')
    endtag = ''
    with open(filename, 'r') as f:
        doc = ''  # initialize current XML doc to empty string
        for line in f:
            doc += line
            endtag = endtag_regex.findall(line) if not endtag else endtag   #is there any endtags?
            if not endtag:
                continue      #if there's no end tag, keep the loop going
            terminate = re.compile('^</{0}>'.format(endtag[0]))    #if there's a end tag, then check whether the doc is finished or not
            if terminate.findall(line):     #if there's a terminator
                yield (doc)                 #export doc and reinitialize a doc
                endtag = ''
                doc = ''

__파일 크기 너무 클테니 두 개로 쪼갬 05~08, 09~12__

In [3]:
date = '20050104'  #initializing date

with open('pat_citation_1.csv', 'wb') as cit:
    writer = csv.writer(cit,quoting=csv.QUOTE_ALL)
    
    while not date=='20090106':
        filename = 'pat_xmls/ipgb'+ date + '.xml'   #making filename
        xml = extract_xml_strings(filename)   #load and split whole xml into app_xmls  
        for doc in xml:
            root = et.fromstring(doc)
            n = root.iter('doc-number')
            pid = n.next().text

            try:
                pid = int(pid)                          
                for cit in root.iter('citation'):   #using iter because of multi-assignee apps 
                    did = cit[0][0]
                    try:
                        cited = cit[1].text              
                        try:
                            if did.find('country').text != 'US':  #if foreign cit, go to next doc
                                break 

                            cid = did.find('doc-number').text
                            if len(cid) > 8:     #if app cit, go to next doc
                                break
                            
                            cit_dum = 1 if cited.split()[-1] == 'examiner' else 0

                            writer.writerow([pid,cid,cit_dum])

                        except AttributeError:         #if other cit, go to next doc
                            break
                        
                    except IndexError:    #if other cit, go to next doc
                        break
                        
            except ValueError:      #if pid != int, go to the next doc
                continue    
                             
        date = dt.datetime.strptime(date,'%Y%m%d')   #Let's move on to next xml
        date += dt.timedelta(weeks=1)
        date = date.strftime('%Y%m%d')

In [4]:
date = '20090106'  #initializing date

with open('pat_citation_2.csv', 'wb') as cit:
    writer = csv.writer(cit,quoting=csv.QUOTE_ALL)
    
    while not date=='20130101':
        filename = 'pat_xmls/ipgb'+ date + '.xml'   #making filename
        xml = extract_xml_strings(filename)   #load and split whole xml into app_xmls  
        for doc in xml:
            root = et.fromstring(doc)
            n = root.iter('doc-number')
            pid = n.next().text

            try:
                pid = int(pid)                          
                for cit in root.iter('citation'):   #using iter because of multi-assignee apps 
                    did = cit[0][0]
                    try:
                        cited = cit[1].text              
                        try:
                            if did.find('country').text != 'US':  #if foreign cit, go to next doc
                                break 

                            cid = did.find('doc-number').text
                            if len(cid) > 8:     #if app cit, go to next doc
                                break
                            
                            cit_dum = 1 if cited.split()[-1] == 'examiner' else 0

                            writer.writerow([pid,cid,cit_dum])

                        except AttributeError:         #if other cit, go to next doc
                            break
                        
                    except IndexError:    #if other cit, go to next doc
                        break
                        
            except ValueError:      #if pid != int, go to the next doc
                continue    
                             
        date = dt.datetime.strptime(date,'%Y%m%d')   #Let's move on to next xml
        date += dt.timedelta(weeks=1)
        date = date.strftime('%Y%m%d')

### examiner or firm 으로 분리

In [1]:
import os
import pandas as pd

os.chdir('E:/apps')
df = pd.read_csv('pat_citation_1.csv')
df.sort_values(by='patent_id',inplace=True)
len(df)

10218430

In [2]:
df1 = df.loc[df['by_examiner']==0]
len(df1)

6884027

In [3]:
del df1['by_examiner']
df1.to_csv('pat_citation_byfirm_1.csv',index=False)

In [4]:
del df1

df = pd.read_csv('pat_citation_2.csv')
df.sort_values(by='patent_id',inplace=True)
len(df)

16823021

In [5]:
df1 = df.loc[df['by_examiner']==0]
len(df1)

12810011

In [6]:
del df1['by_examiner']
df1.to_csv('pat_citation_byfirm_2.csv',index=False)